In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

In [3]:
plt.rcParams['figure.figsize'] = (15, 8)

In [4]:
base_db = pd.read_csv("use_db_sabato14.csv")
base_db.head()

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,10108-64-2,S,48.0,T,46.900,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
1,10108-64-2,S,24.0,T,200.000,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Fundulus,heteroclitus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
2,10108-64-2,F,96.0,T,0.001,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
3,10108-64-2,S,96.0,T,15.900,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Cyprinodon,variegatus,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945
4,10108-64-2,F,694.0,T,0.030,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.0,0.0,0.096146,0.666667,-5.9945


## Simple classification 
Considering a threshold of 1mg/L of mortality

In [5]:
base_db

,test_cas,exposure_type,obs_duration_mean,conc1_type,conc1_mean,class,tax_order,family,genus,species,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP
0,10108-64-2,S,48.0,T,46.900,Actinopterygii,Cypriniformes,Cyprinidae,Carassius,auratus,0.273831,3,0,0,0.000000,0.0,0.096146,0.666667,-5.9945
1,10108-64-2,S,24.0,T,200.000,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Fundulus,heteroclitus,0.273831,3,0,0,0.000000,0.0,0.096146,0.666667,-5.9945
2,10108-64-2,F,96.0,T,0.001,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.000000,0.0,0.096146,0.666667,-5.9945
3,10108-64-2,S,96.0,T,15.900,Actinopterygii,Cyprinodontiformes,Cyprinodontidae,Cyprinodon,variegatus,0.273831,3,0,0,0.000000,0.0,0.096146,0.666667,-5.9945
4,10108-64-2,F,694.0,T,0.030,Actinopterygii,Salmoniformes,Salmonidae,Oncorhynchus,mykiss,0.273831,3,0,0,0.000000,0.0,0.096146,0.666667,-5.9945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53175,103-05-9,F,96.0,A,66.400,Actinopterygii,Cypriniformes,Cyprinidae,Pimephales,promelas,0.384370,0,0,0,0.522111,1.0,0.086066,1.166667,2.3901
53176,14008-58-3,AQUA,28.0,F,0.056,Actinopterygii,Cypriniformes,Cyprinidae,Notropis,sp.,0.473649,0,0,2,0.576717,1.0,0.115904,1.437500,1.3211
53177,52316-55-9,S,96.0,A,257.000,Actinopterygii,Perciformes,Centrarchidae,Lepomis,macrochirus,0.490837,2,0,2,0.609799,2.0,0.150327,1.368421,0.8126
53178,499-83-2,F,96.0,A,322.000,Actinopterygii,Cypriniformes,Cyprinidae,Pimephales,promelas,0.384370,0,0,2,0.522111,1.0,0.087591,1.083333,0.4780


In [6]:
base_db = base_db[base_db.obs_duration_mean.isin([24, 48, 72, 96])]

In [7]:
to_drop = base_db[base_db.bonds_number.isnull()].index

In [8]:
base_db = base_db.drop(to_drop,axis=0)

In [9]:
base_db.index.size

45775

In [10]:
base_db.columns

Index(['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species',
       'atom_number', 'alone_atom_number', 'tripleBond', 'doubleBond',
       'bonds_number', 'ring_numer', 'Mol', 'MorganDensity', 'LogP'],
      dtype='object')

In [11]:
use_db = base_db.copy()
use_db = use_db[['test_cas', 'exposure_type', 'obs_duration_mean', 'conc1_type',
       'conc1_mean', 'class', 'tax_order', 'family', 'genus', 'species','atom_number', 'alone_atom_number', 'tripleBond',
       'doubleBond', 'bonds_number', 'ring_numer',
       'Mol', 'MorganDensity', 'LogP']]

In [12]:
use_db.index.size

45775

# Repeated experiments

In [13]:
db_species = use_db[["species", 'class', 'tax_order', 'family', 'genus']]

In [14]:
use_db = use_db.groupby(by=["test_cas", "species", "conc1_type", "exposure_type", "obs_duration_mean",]).agg("median").reset_index()
use_db.index.size

23332

In [15]:
db_species = db_species.groupby("species").first()

In [16]:
use_db = use_db.merge(db_species, on="species")

# 1a
let's simply try with a get dummies

In [18]:
X = use_db.copy()
X = X[[
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species',
    'class', 'tax_order', 'family', 'genus',
 'atom_number', 
    'alone_atom_number', 
    'tripleBond',
       'doubleBond',
 'bonds_number',
  'ring_numer',
  'Mol', 
  'MorganDensity', 'LogP']]
X = pd.get_dummies(X)

In [42]:
X

,obs_duration_mean,atom_number,alone_atom_number,tripleBond,doubleBond,bonds_number,ring_numer,Mol,MorganDensity,LogP,...,genus_Tribolodon,genus_Trichogaster,genus_Tridentiger,genus_Umbra,genus_Vimba,genus_Wallago,genus_Xiphophorus,genus_Xyrauchen,genus_Zacco,genus_Zosterisessor
0,48.0,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,...,0,0,0,0,0,0,0,0,0,0
1,96.0,0.317908,2,0,1,0.488106,1.0,0.082505,1.300000,2.2482,...,0,0,0,0,0,0,0,0,0,0
2,96.0,0.317908,2,0,1,0.488106,1.0,0.072092,1.300000,1.1770,...,0,0,0,0,0,0,0,0,0,0
3,48.0,0.317908,2,0,1,0.488106,1.0,0.072622,1.300000,1.3004,...,0,0,0,0,0,0,0,0,0,0
4,96.0,0.217006,0,0,0,0.447258,1.0,0.065856,1.125000,2.4254,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23327,96.0,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,...,0,0,0,0,0,0,0,0,0,0
23328,24.0,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,...,0,0,0,0,0,0,0,0,0,0
23329,48.0,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,...,0,0,0,0,0,0,0,0,0,0
23330,96.0,0.410746,0,0,1,0.537196,1.0,0.115790,1.461538,2.4568,...,0,0,0,0,0,0,0,0,0,0


# 1b
let's simply try with a get dummies but keeping only categorical values

In [15]:
X = use_db.copy()
X = X[[
 'test_cas',
 'exposure_type',
 'conc1_type',
 'species', 
 'genus']]
X = pd.get_dummies(X)

# 2
let's try another specification (we have to specify the categorical columns)

In [8]:
X = use_db.copy()
X = X[[
 'test_cas',
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species', 
 'genus']]

enc = OrdinalEncoder(dtype=int)
enc.fit(X[['test_cas',
             'exposure_type',
             'conc1_type',
             'species', 
             'genus']])
X[['test_cas','exposure_type','conc1_type','species', 'genus']] = enc.transform(X[['test_cas','exposure_type','conc1_type','species','genus']]) + 1

# 3
and what about keeping only the categorical values?

In [20]:
X = use_db.copy()
X = X[['test_cas','exposure_type','conc1_type','species', 'genus']]

enc = OrdinalEncoder(dtype=int)
enc.fit(X[['test_cas',
             'exposure_type',
             'conc1_type',
             'species', 
             'genus']])
X[['test_cas','exposure_type','conc1_type','species', 'genus']] = enc.transform(X[['test_cas','exposure_type','conc1_type','species','genus']]) + 1

Setting threshold

In [20]:
y = use_db[["conc1_mean"]].copy().values
y = np.where(y > 1, 1, 0)

Checking if the splitting is balanced:

In [21]:
print(len(y[y==0]), len(y[y==1]))

8566 14766


In [33]:
def scores_cat(x):
    if (x < 10**-1):
        return 5
    # elif ((x>=10**-2) and (x<10**-1)):
    #     return 6
    elif ((x>=10**-1) and (x<10**0)):
        return 4
    elif ((x>=10**0) and (x<10**1)):
        return 3
    elif ((x>=10**1) and (x<10**2)):
        return 2
    # elif ((x>=10**2) and (x<10**3)):
    #     return 2
    else:
        return 1

In [34]:
y = use_db["conc1_mean"].copy().values
y = pd.Series(y).apply(lambda x: scores_cat(x)).values

### KNN

In [35]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.33, random_state=42)

In [23]:
neigh = KNeighborsClassifier() #metric="hamming"
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [24]:
accuracy_score(y_test, y_pred)

0.7912987012987013

### KNN multi

In [36]:
X_train, X_test, y_train, y_test = train_test_split(
                                        X, y, test_size=0.33, random_state=42)

In [37]:
neigh = KNeighborsClassifier() #metric="hamming"
neigh.fit(X_train, y_train.ravel())
y_pred = neigh.predict(X_test)

In [38]:
accuracy_score(y_test, y_pred)

0.4737662337662338

### Random Forest

In [26]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train.ravel())
y_pred = forest.predict(X_test)

C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [27]:
accuracy_score(y_test, y_pred)

0.8754545454545455

### Random Forest multi

In [39]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train.ravel())
y_pred = forest.predict(X_test)

C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [40]:
accuracy_score(y_test, y_pred)

0.6768831168831169

## Logistic regression

In [32]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=13).fit(X_train, y_train.ravel())
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7694805194805194

## Logistic regression multi

In [41]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=13).fit(X_train, y_train.ravel())
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Manuel\Anaconda3\envs\ml\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.44805194805194803

### CV on KNN (best result so far)

In [16]:
grid = GridSearchCV(neigh, param_grid={"n_neighbors": list(range(1, 15))}, scoring="accuracy", cv=5, verbose=1, n_jobs=-2)
grid.fit(X_train, y_train.ravel());

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-2)]: Done  70 out of  70 | elapsed:  1.7min finished


In [17]:
grid.best_params_

{'n_neighbors': 1}

Best results is **k=1**

In [21]:
grid.best_score_

0.9015576816636356

In [22]:
y_pred = grid.predict(X_test)

In [23]:
accuracy_score(y_test, y_pred)

0.9041630993150684

#### 90% of accuracy with KNN, k=1, threshold 1 mg/L

## Simple regression

In [24]:
X = use_db.copy()

In [25]:
X = X[[
 'test_cas',
 'exposure_type',
 'obs_duration_mean',
 'conc1_type',
 'species', 
 'genus']]

Categorical variables with get_dummies:

In [26]:
X = pd.get_dummies(X)

In [27]:
y = base_db[["conc1_mean"]].copy()

CV on ridge

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.33, random_state=42)

reg = RidgeCV(alphas=np.logspace(-6, 6, 13))
reg.fit(X_train, y_train);

In [44]:
reg.alpha_

100.0

In [45]:
y_pred = reg.predict(X_test)

In [46]:
mean_squared_error(y_test, y_pred)

221955983.9236199

In [47]:
y_pred_train = reg.predict(X_train)
mean_squared_error(y_train, y_pred_train)

282426813.18152875

As expected, **the loss is huge**